In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
from tqdm import tqdm
import psutil
import zipfile 
import shutil

In [2]:
current_dir = os.getcwd()
print(current_dir)

C:\Users\zkr_l\Desktop\Trading\CW


Unzip data and define paths to data:

In [3]:
path_to_zip_file = f"{current_dir}/data/binSamples.zip"
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(current_dir)

path_to_zip_file = f"{current_dir}/data/fillSamples.zip"
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(current_dir)

In [4]:
year = "2019"
bin_sample_path = f"{current_dir}/data/binSamples/"
fill_sample_path = f"{current_dir}/data/fillSamples/"
result_path = f"{current_dir}/data/Results/"

Prepare dataframes:

In [5]:
stock_info_dfs = []

for month_int in tqdm(range(1, 13)):
    # read data by months
    month = "%02d" % month_int
    filename = f"bin{year}{month}.csv"
    bin_sample_df = pd.read_csv(bin_sample_path + filename)
    
    # generate traded volume dataframe indexed by stock and date
    traded_volume_df = bin_sample_df[["stock", "date", "trade", "time"]].pivot(index=["stock", "date"], columns="time", values="trade").fillna(0).astype(int)
    filename = f"inter_results_trade_volume_df_{year}{month}.csv"
    traded_volume_df.to_csv(result_path + filename)

    # generate mid price dataframe (at the end of the bin window) indexed by stock and date
    px_df = bin_sample_df[["stock", "date", "midEnd", "time"]].pivot(index=["stock", "date"], columns="time", values="midEnd").fillna(method="ffill", axis="columns").fillna(method="bfill", axis="columns")
    filename = f"inter_results_px_df_{year}{month}.csv"
    px_df.to_csv(result_path + filename)

    # generate spread dataframe (at start of bin window) indexed by stock and date
    spread_df = bin_sample_df[["stock", "date", "spread", "time"]].pivot(index=["stock", "date"], columns="time", values="spread").fillna(method="ffill", axis="columns").fillna(method="bfill", axis="columns")
    filename = f"inter_results_spread_df_{year}{month}.csv"
    spread_df.to_csv(result_path + filename)

    # generate daily price return volatility std and traded volume dataframe
    monthly_stock_info_df = pd.DataFrame({
        # Compute the price return volatility as a standard deviation row-wise
        "px_vol": px_df.pct_change(1, axis="columns").std(axis="columns"),
        # Compute the total daily traded volume as the sum of all volumes in either direction using abp().sum over rows (axis columns)
        "volume": traded_volume_df.abs().sum(axis="columns"),
    }).reset_index()
    stock_info_dfs.append(monthly_stock_info_df.copy())

stock_info_df = pd.concat(stock_info_dfs).reset_index()
filename = f"inter_results_stock_info_df_{year}all.csv"
stock_info_df.to_csv(result_path + filename)

  0%|          | 0/12 [00:00<?, ?it/s]C:\Users\zkr_l\AppData\Local\Temp\ipykernel_21276\1933955430.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  px_df = bin_sample_df[["stock", "date", "midEnd", "time"]].pivot(index=["stock", "date"], columns="time", values="midEnd").fillna(method="ffill", axis="columns").fillna(method="bfill", axis="columns")
C:\Users\zkr_l\AppData\Local\Temp\ipykernel_21276\1933955430.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  spread_df = bin_sample_df[["stock", "date", "spread", "time"]].pivot(index=["stock", "date"], columns="time", values="spread").fillna(method="ffill", axis="columns").fillna(method="bfill", axis="columns")
  8%|▊         | 1/12 [00:17<03:17, 17.99s/it]C:\Users\zkr_l\AppData\Local\Temp\ipykernel_21276\1933955430.py:15: FutureWarning: DataFrame.filln

Collecting pairs of months together to take 20-day rolling average.

In [6]:
stock_info_dfs = []

for month_int in tqdm(range(1, 12)):
    # read data by months
    month = "%02d" % month_int
    filename = f"bin{year}{month}.csv"
    bin_sample_df = pd.read_csv(bin_sample_path + filename)
    
    month2 = "%02d" % (month_int+1)
    filename2 = f"bin{year}{month2}.csv"
    bin_sample_df2 = pd.read_csv(bin_sample_path + filename2)
    
    traded_volume_df = bin_sample_df[["stock", "date", "trade", "time"]].pivot(index=["stock", "date"], columns="time", values="trade").fillna(0).astype(int)
    traded_volume_df2 = bin_sample_df2[["stock", "date", "trade", "time"]].pivot(index=["stock", "date"], columns="time", values="trade").fillna(0).astype(int)
    
    px_df = bin_sample_df[["stock", "date", "midEnd", "time"]].pivot(index=["stock", "date"], columns="time", values="midEnd").fillna(method="ffill", axis="columns").fillna(method="bfill", axis="columns")
    px_df2 = bin_sample_df2[["stock", "date", "midEnd", "time"]].pivot(index=["stock", "date"], columns="time", values="midEnd").fillna(method="ffill", axis="columns").fillna(method="bfill", axis="columns")
    
    
    # generate daily price return volatility std and traded volume dataframe
    monthly_stock_info_df = pd.DataFrame({
        # Compute the price return volatility as a standard deviation row-wise
        "px_vol": px_df.pct_change(1, axis="columns").std(axis="columns"),
        # Compute the total daily traded volume as the sum of all volumes in either direction using abp().sum over rows (axis columns)
        "volume": traded_volume_df.abs().sum(axis="columns"),
    }).reset_index()
    monthly_stock_info_df2 = pd.DataFrame({
        # Compute the price return volatility as a standard deviation row-wise
        "px_vol": px_df2.pct_change(1, axis="columns").std(axis="columns"),
        # Compute the total daily traded volume as the sum of all volumes in either direction using abp().sum over rows (axis columns)
        "volume": traded_volume_df2.abs().sum(axis="columns"),
    }).reset_index()
    
    stock_info_dfs = [monthly_stock_info_df.copy(), monthly_stock_info_df2.copy()]
    stock_info_df = pd.concat(stock_info_dfs).reset_index()
    filename = f"stock_info_df_{year}{month}{month2}.csv"
    stock_info_df.to_csv(result_path + filename)

  0%|          | 0/11 [00:00<?, ?it/s]C:\Users\zkr_l\AppData\Local\Temp\ipykernel_21276\3614617416.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  px_df = bin_sample_df[["stock", "date", "midEnd", "time"]].pivot(index=["stock", "date"], columns="time", values="midEnd").fillna(method="ffill", axis="columns").fillna(method="bfill", axis="columns")
C:\Users\zkr_l\AppData\Local\Temp\ipykernel_21276\3614617416.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  px_df2 = bin_sample_df2[["stock", "date", "midEnd", "time"]].pivot(index=["stock", "date"], columns="time", values="midEnd").fillna(method="ffill", axis="columns").fillna(method="bfill", axis="columns")
  9%|▉         | 1/11 [00:17<02:59, 17.93s/it]C:\Users\zkr_l\AppData\Local\Temp\ipykernel_21276\3614617416.py:16: FutureWarning: DataFrame.fillna 

In [7]:
bin_sample_df

,date,time,stock,trade,orderFlow,hidden,auction,mid,midEnd,spread,effSpread,lobImb,effLobImb,trdLiq,ofLiq,depth,nbEvents,nbHidden,nbTrades
0,2019-11-01,09:30:00,A,0,84,0,0,76.335,76.420,0.225,NaN,0.739130,NaN,0,316,157.5000,2,0,0
1,2019-11-01,09:30:10,A,0,36,0,0,76.340,76.340,0.210,NaN,0.333333,NaN,0,36,3.0000,1,0,0
2,2019-11-01,09:30:30,A,0,34,65,0,76.350,76.395,0.200,NaN,0.333333,NaN,0,34,18.0000,2,1,0
3,2019-11-01,09:30:40,A,0,136,0,0,76.395,76.395,0.155,NaN,0.942857,NaN,0,136,35.0000,1,0,0
4,2019-11-01,09:30:50,A,0,-5,0,0,76.395,76.395,0.155,NaN,0.987952,NaN,0,5,166.0000,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946539,2019-11-29,12:59:10,ARE,0,105,16,0,162.625,162.600,0.085,NaN,0.000000,NaN,0,155,180.0000,4,1,0
1946540,2019-11-29,12:59:20,ARE,-32,0,0,0,162.610,162.660,0.020,0.050000,0.877301,-0.183673,32,722,222.8000,5,0,1
1946541,2019-11-29,12:59:30,ARE,-140,124,0,0,162.605,162.645,0.065,0.025000,0.203187,-0.061033,140,1946,234.6000,5,0,1
1946542,2019-11-29,12:59:40,ARE,-354,300,16,0,162.640,162.700,0.020,0.020000,0.818182,0.069519,354,2322,275.3333,6,1,1


In [8]:
traded_volume_df

time              09:30:00  09:30:10  09:30:20  09:30:30  09:30:40  09:30:50  \
stock date                                                                     
A     2019-11-01         0         0         0         0         0         0   
      2019-11-04      -134        91         0         0         0         0   
      2019-11-05         0         0         0         0         0         0   
      2019-11-06         0         0         0         0         0         0   
      2019-11-07         0         0      -108         0         0         0   
...                    ...       ...       ...       ...       ...       ...   
ARE   2019-11-22         0         0         0         0         0         0   
      2019-11-25         0         0         0         0         0         0   
      2019-11-26         0         0         0         0       105         0   
      2019-11-27      -167      -300         0         0         0         0   
      2019-11-29         0         0         0         0         0         0   

time              09:31:00  09:31:10  09:31:20  09:31:30  ...  15:58:30  \
stock date                                                ...             
A     2019-11-01         0         0         0         0  ...       275   
      2019-11-04         0         0         0         0  ...         0   
      2019-11-05         0         0        -4         0  ...      -207   
      2019-11-06         0      -345         2         0  ...       100   
      2019-11-07         0         0         0         0  ...         0   
...                    ...       ...       ...       ...  ...       ...   
ARE   2019-11-22         0         0         0         0  ...         0   
      2019-11-25         0         0         0       132  ...      -205   
      2019-11-26         0         0         0         0  ...      -152   
      2019-11-27         0         0         0         0  ...      -107   
      2019-11-29         0       107         0         0  ...         0   

time              15:58:40  15:58:50  15:59:00  15:59:10  15:59:20  15:59:30  \
stock date                                                                     
A     2019-11-01       433         8      -551      -400       635       300   
      2019-11-04       200       100      -495     -1493         0      -200   
      2019-11-05      -149       246       207        50         0     -2543   
      2019-11-06      -249      -459       189      -317       108       104   
      2019-11-07       191       100        23       -26       136       100   
...                    ...       ...       ...       ...       ...       ...   
ARE   2019-11-22      -171         0       121       -69       200       475   
      2019-11-25         0       102       200       351       111       707   
      2019-11-26       -96         0       253        59      -105       146   
      2019-11-27       105        99      -106      -142       202       -24   
      2019-11-29         0         0         0         0         0         0   

time              15:59:40  15:59:50  16:00:00  
stock date                                      
A     2019-11-01       152      -260         0  
      2019-11-04         0      -942         0  
      2019-11-05      -327      1057         0  
      2019-11-06         0       462         0  
      2019-11-07      -100       -59         0  
...                    ...       ...       ...  
ARE   2019-11-22      -406       -71         0  
      2019-11-25       -94        14         0  
      2019-11-26       -46         5         0  
      2019-11-27      -284        38         0  
      2019-11-29         0         0         0  

[980 rows x 2341 columns]

In [9]:
px_df

time              09:30:00  09:30:10  09:30:20  09:30:30  09:30:40  09:30:50  \
stock date                                                                     
A     2019-11-01    76.420    76.340    76.340    76.395    76.395    76.395   
      2019-11-04    77.670    77.700    77.670    77.670    77.670    77.670   
      2019-11-05    76.895    76.770    76.785    76.790    76.790    76.810   
      2019-11-06    75.810    75.660    75.705    75.705    75.705    75.705   
      2019-11-07    76.370    76.240    76.300    76.180    76.095    76.095   
...                    ...       ...       ...       ...       ...       ...   
ARE   2019-11-22   159.885   159.395   159.215   159.215   159.240   159.260   
      2019-11-25   159.925   159.925   159.925   159.925   159.925   159.925   
      2019-11-26   160.800   160.805   160.905   160.960   160.975   161.230   
      2019-11-27   161.600   161.090   160.825   160.825   160.825   160.820   
      2019-11-29   162.730   162.730   162.765   162.765   162.765   162.765   

time              09:31:00  09:31:10  09:31:20  09:31:30  ...  15:58:30  \
stock date                                                ...             
A     2019-11-01    76.395    76.395    76.395    76.405  ...    76.990   
      2019-11-04    77.670    77.670    77.670    77.680  ...    76.785   
      2019-11-05    76.795    76.805    76.780    76.735  ...    75.615   
      2019-11-06    75.705    75.670    75.665    75.665  ...    75.795   
      2019-11-07    76.200    76.200    76.200    76.200  ...    75.805   
...                    ...       ...       ...       ...  ...       ...   
ARE   2019-11-22   159.270   159.245   159.205   159.205  ...   159.560   
      2019-11-25   159.925   159.925   159.980   160.060  ...   160.505   
      2019-11-26   161.235   161.190   161.235   161.235  ...   161.105   
      2019-11-27   160.820   160.820   160.820   160.795  ...   162.485   
      2019-11-29   162.765   162.765   162.765   162.765  ...   162.585   

time              15:58:40  15:58:50  15:59:00  15:59:10  15:59:20  15:59:30  \
stock date                                                                     
A     2019-11-01    77.000    77.030    77.065    77.035    76.965    76.985   
      2019-11-04    76.800    76.805    76.815    76.790    76.780    76.760   
      2019-11-05    75.575    75.590    75.620    75.600    75.590    75.570   
      2019-11-06    75.795    75.780    75.760    75.790    75.780    75.770   
      2019-11-07    75.810    75.810    75.835    75.840    75.840    75.860   
...                    ...       ...       ...       ...       ...       ...   
ARE   2019-11-22   159.560   159.515   159.605   159.610   159.620   159.630   
      2019-11-25   160.475   160.475   160.485   160.535   160.675   160.720   
      2019-11-26   161.125   161.075   161.075   161.170   161.235   161.250   
      2019-11-27   162.485   162.485   162.515   162.410   162.340   162.350   
      2019-11-29   162.585   162.585   162.585   162.585   162.585   162.585   

time              15:59:40  15:59:50  16:00:00  
stock date                                      
A     2019-11-01    76.990    76.985    76.985  
      2019-11-04    76.760    76.755    76.755  
      2019-11-05    75.495    75.545    75.545  
      2019-11-06    75.785    75.775    75.775  
      2019-11-07    75.855    75.900    75.900  
...                    ...       ...       ...  
ARE   2019-11-22   159.695   159.650   159.650  
      2019-11-25   160.670   160.645   160.645  
      2019-11-26   161.240   161.250   161.250  
      2019-11-27   162.340   162.295   162.295  
      2019-11-29   162.585   162.585   162.585  

[980 rows x 2341 columns]

In [10]:
spread_df

time              09:30:00  09:30:10  09:30:20  09:30:30  09:30:40  09:30:50  \
stock date                                                                     
A     2019-12-02     0.320     0.095     0.105     0.090     0.095     0.090   
      2019-12-03     0.310     0.240     0.135     0.155     0.090     0.170   
      2019-12-04     0.235     0.135     0.080     0.060     0.075     0.080   
      2019-12-05     0.130     0.130     0.130     0.090     0.115     0.170   
      2019-12-06     0.335     0.140     0.140     0.110     0.110     0.105   
...                    ...       ...       ...       ...       ...       ...   
ARE   2019-12-20     0.530     0.530     0.485     0.515     0.615     0.410   
      2019-12-23     0.660     0.295     0.210     0.170     0.165     0.165   
      2019-12-24     0.590     0.590     0.590     0.350     0.350     0.350   
      2019-12-26     0.700     0.700     0.700     0.700     0.235     0.235   
      2019-12-27     0.730     0.475     0.460     0.460     0.460     0.385   

time              09:31:00  09:31:10  09:31:20  09:31:30  ...  15:58:30  \
stock date                                                ...             
A     2019-12-02     0.075     0.075     0.055     0.050  ...     0.015   
      2019-12-03     0.305     0.305     0.330     0.150  ...     0.005   
      2019-12-04     0.060     0.040     0.050     0.050  ...     0.010   
      2019-12-05     0.165     0.165     0.165     0.160  ...     0.005   
      2019-12-06     0.105     0.105     0.105     0.085  ...     0.005   
...                    ...       ...       ...       ...  ...       ...   
ARE   2019-12-20     0.410     0.210     0.210     0.165  ...     0.015   
      2019-12-23     0.150     0.150     0.150     0.150  ...     0.015   
      2019-12-24     0.350     0.345     0.345     0.335  ...     0.075   
      2019-12-26     0.225     0.225     0.225     0.195  ...     0.025   
      2019-12-27     0.385     0.325     0.325     0.325  ...     0.010   

time              15:58:40  15:58:50  15:59:00  15:59:10  15:59:20  15:59:30  \
stock date                                                                     
A     2019-12-02     0.010     0.010     0.015     0.020     0.015     0.010   
      2019-12-03     0.010     0.015     0.010     0.015     0.010     0.015   
      2019-12-04     0.020     0.020     0.025     0.020     0.010     0.010   
      2019-12-05     0.005     0.005     0.015     0.015     0.010     0.005   
      2019-12-06     0.005     0.005     0.005     0.015     0.005     0.015   
...                    ...       ...       ...       ...       ...       ...   
ARE   2019-12-20     0.015     0.020     0.010     0.045     0.030     0.050   
      2019-12-23     0.015     0.010     0.005     0.025     0.015     0.020   
      2019-12-24     0.075     0.075     0.075     0.075     0.075     0.075   
      2019-12-26     0.025     0.020     0.030     0.030     0.040     0.040   
      2019-12-27     0.020     0.015     0.005     0.020     0.020     0.025   

time              15:59:40  15:59:50  16:00:00  
stock date                                      
A     2019-12-02     0.010     0.010     0.010  
      2019-12-03     0.010     0.020     0.020  
      2019-12-04     0.020     0.020     0.020  
      2019-12-05     0.010     0.010     0.010  
      2019-12-06     0.005     0.005     0.005  
...                    ...       ...       ...  
ARE   2019-12-20     0.015     0.030     0.030  
      2019-12-23     0.020     0.030     0.030  
      2019-12-24     0.075     0.075     0.075  
      2019-12-26     0.020     0.035     0.035  
      2019-12-27     0.020     0.020     0.020  

[931 rows x 2341 columns]

In [11]:
monthly_stock_info_df 

,stock,date,px_vol,volume
0,A,2019-11-01,0.000192,168255
1,A,2019-11-04,0.000181,101881
2,A,2019-11-05,0.000234,163436
3,A,2019-11-06,0.000224,121174
4,A,2019-11-07,0.000207,95653
...,...,...,...,...
975,ARE,2019-11-22,0.000223,78377
976,ARE,2019-11-25,0.000194,57685
977,ARE,2019-11-26,0.000257,92637
978,ARE,2019-11-27,0.000194,59014


In [12]:
stock_info_df

,index,stock,date,px_vol,volume
0,0,A,2019-11-01,0.000192,168255
1,1,A,2019-11-04,0.000181,101881
2,2,A,2019-11-05,0.000234,163436
3,3,A,2019-11-06,0.000224,121174
4,4,A,2019-11-07,0.000207,95653
...,...,...,...,...,...
1906,926,ARE,2019-12-20,0.000225,57796
1907,927,ARE,2019-12-23,0.000112,45866
1908,928,ARE,2019-12-24,0.000135,17555
1909,929,ARE,2019-12-26,0.000138,24736


In [13]:
for month_int in tqdm(range(1, 13)):
    month = '%02d' % month_int
    filename = f"inter_results_px_df_{year}{month}.csv"
    px_df = pd.read_csv(result_path+filename).set_index(["stock","date"])
    filename = f"inter_results_trade_volume_df_{year}{month}.csv"
    traded_volume_df = pd.read_csv(result_path+filename).set_index(["stock","date"])
    filename = f"inter_results_spread_df_{year}{month}.csv"
    spread_df = pd.read_csv(result_path+filename).set_index(["stock","date"])

    filename = f"adjusted_prices_{year}{month}.csv"
    px_df = px_df + np.multiply(traded_volume_df.abs(), spread_df) / 2
    px_df.to_csv(result_path+filename)

100%|██████████| 12/12 [01:10<00:00,  5.89s/it]


In [14]:
px_df

09:30:00  09:30:10  09:30:20  09:30:30  09:30:40  09:30:50  \
stock date                                                                     
A     2019-12-02   80.7500    80.765    80.785    81.445    80.715    80.695   
      2019-12-03  188.2400    79.630    86.335    79.610    81.875    79.540   
      2019-12-04   80.2450    80.330    80.310    83.305    97.105    80.245   
      2019-12-05   80.8600    80.860    80.860   107.990    80.765    80.725   
      2019-12-06   99.3175    82.110    82.110    87.695    82.140    82.140   
...                    ...       ...       ...       ...       ...       ...   
ARE   2019-12-20  160.2850   160.285   159.885   159.875   329.275   159.460   
      2019-12-23  159.5650   159.655   162.385   159.540   159.540   159.540   
      2019-12-24  159.3250   159.325   165.815   159.570   159.570   159.570   
      2019-12-26  160.8300   160.830   160.830   160.830   160.435   160.435   
      2019-12-27  160.5700   160.720   160.695   160.695   160.695   160.675   

                  09:31:00  09:31:10  09:31:20  09:31:30  ...  15:58:30  \
stock date                                                ...             
A     2019-12-02    80.675    80.675    80.675    80.680  ...   80.3800   
      2019-12-03    79.495    79.495    79.375    79.355  ...   82.6825   
      2019-12-04    80.345    82.240    80.225    80.230  ...   81.0500   
      2019-12-05    80.685    80.685    80.600    80.690  ...   81.4850   
      2019-12-06    82.135    82.135    82.135    82.180  ...   82.3775   
...                    ...       ...       ...       ...  ...       ...   
ARE   2019-12-20   159.460   159.660   159.660   159.660  ...  160.8550   
      2019-12-23   161.500   159.720   159.720   159.725  ...  159.2325   
      2019-12-24   159.570   159.570   159.575   159.575  ...  160.1550   
      2019-12-26   160.445   160.445   160.445   160.445  ...  159.8450   
      2019-12-27   160.675   160.595   160.595   160.595  ...  160.9450   

                  15:58:40  15:58:50  15:59:00  15:59:10  15:59:20  15:59:30  \
stock date                                                                     
A     2019-12-02   80.3700   81.3600   85.5925    81.410   81.1750    81.910   
      2019-12-03   82.5200   81.5450   80.5700    82.315   80.0600    81.550   
      2019-12-04   82.6550   80.9250   80.9550    80.915   82.4250    81.430   
      2019-12-05   81.7325   82.0225   83.0200    81.850   81.7800    81.790   
      2019-12-06   82.1650   82.4025   83.0650    83.745   84.5750    91.160   
...                    ...       ...       ...       ...       ...       ...   
ARE   2019-12-20  159.2800  163.2750  160.4250   159.330  160.8300   161.180   
      2019-12-23  161.7075  159.2300  160.4025   161.565  159.5925   159.245   
      2019-12-24  160.1550  160.1550  160.1550   160.155  160.1550   160.155   
      2019-12-26  159.8450  160.7500  162.8500   159.945  159.9300   159.945   
      2019-12-27  161.6050  160.4000  160.8300   160.355  160.4800   160.355   

                  15:59:40  15:59:50  16:00:00  
stock date                                      
A     2019-12-02   80.8300    83.360    80.360  
      2019-12-03   80.6050    88.865    80.065  
      2019-12-04   81.9850    82.010    80.930  
      2019-12-05   83.6750    82.505    81.515  
      2019-12-06   84.9950    82.230    82.225  
...                    ...       ...       ...  
ARE   2019-12-20  161.2525   159.255   159.225  
      2019-12-23  159.2800   159.465   159.270  
      2019-12-24  160.1550   160.155   160.155  
      2019-12-26  160.9650   160.825   159.950  
      2019-12-27  160.7400   162.400   160.400  

[931 rows x 2341 columns]